In [1]:
%load_ext autoreload
%autoreload 2
import seaborn as sns
import metapack as mp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display

%matplotlib inline
sns.set_context('notebook')
mp.jupyter.init()


In [2]:
pkg = mp.jupyter.open_package()
#pkg = mp.jupyter.open_source_package()
pkg.set_sys_path()
from pylib import *
pkg

`isr.umich.edu-psid-inequality-3.2.1` Last Update: 2023-06-16T00:20:09

__



## Updating Data

To update the data, use the data extraction tool at the (PSID website. )[https://simba.isr.umich.edu/Cart/co.aspx], adding any new variables to your data cart. The (variable entry text box)[https://simba.isr.umich.edu/DC/l.aspx] is a handy way to copy the existing varibles into a new cart.  When you checkout your data cart, ( click on the cart icon ) fetch your dataset with these options: 

* Codebook Type: XML
* Data output type: dBase Data File, create compressed file

Download the ZIP file ( you might have to recompress it if your browser unpacks it ) and move it into the ``data`` directory. Set the reference to the new ZIP file, then re-build the package.
## Documentation Links

* [PSID Homepage](https://psidonline.isr.umich.edu/default.aspx) 

 

## Contacts

* **Wrangler** [Eric Busboom](mailto:eric@civicknowledge.com), [Civic Knowledge](http://civicknowledge.com)

## Resources

* **[variables](data/variables.csv)**. All PSID variables, with group infor for connecting variables across years.
* **[cmp9919_labels](data/cmp9919_labels.csv)**. Categorical value labels
* **[long_wealth_labels](data/long_wealth_labels.csv)**. Categorical value labels
* **[cmp9919_dd](data/cmp9919_dd.csv)**. Data dictionary with information about variables in this collection, extracted from XML
* **[long_wealth_dd](data/long_wealth_dd.csv)**. Data dictionary with information about variables in this collection, extracted from XML
* **[cmp9919](data/cmp9919.csv)**. PSID, Inequality extract
* **[long_wealth](data/long_wealth.csv)**. Longitudinal Wealth

## References
<ul><li> <strong><a href="https://fred.stlouisfed.org/graph/fredgraph.csv?&id=CPIAUCSL">cpi</a></strong>. Consumer Price Index, from FRED series CPIAUCSL</li><li> <strong><a href="https://psidonline.isr.umich.edu/help/xyr/psid.xlsx">variables</a></strong>. Variable descriptions, in a cross-year format.</li><li> <strong>cmp9919_source</string>, <em>data/J321148.zip</em>. Source file for comparing 1999 to 2019</li><li> <strong>long_wealth_source</string>, <em>data/J321388.zip</em>. Source data package for Longitudinal Wealth Study</li><ul>

In [3]:
r = pkg.resource('long_wealth')
df = pkg.resource('long_wealth').dataframe()
df.head()

,pid,year,race,er30001,er30002,er30000,er33501,er33503,er13001,er33502,...,er16462,er16518,s405,s419,s420,s415,s411,s413,s417,s416
0,4001,2001,NaN,4,1,3,0,0,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4003,2001,1.0,4,3,3,96,10,6.0,1,...,50880.0,19.812,50.0,0.0,90000.0,0.0,15000.0,8000.0,113050.0,23050.0
2,4004,2001,1.0,4,4,3,5987,10,6.0,1,...,50000.0,30.397,10000.0,35000.0,68000.0,0.0,10000.0,20000.0,493000.0,425000.0
3,4006,2001,1.0,4,6,3,5599,22,6.0,2,...,92000.0,19.566,4000.0,0.0,13000.0,0.0,0.0,6000.0,23000.0,10000.0
4,4007,2001,1.0,4,7,3,7091,10,6.0,1,...,55616.0,24.971,15000.0,0.0,17000.0,0.0,0.0,9000.0,41000.0,24000.0


In [4]:
m = Metadata(r)
m.lookup('ER33616').T

,241
type,INDIVIDUAL
category,EDUCATION
text,01>EDUCATION:\n02>Completed Education\n03>grad...
head_wife,NaN
var_count,40
group,64
year,2001
name,ER33616
path,/education:/completed education/grades completed:


In [27]:
r = pkg.resource('long_wealth')
cat = r.cat

# These have missing/imputed values that make them categories, but they really should be numeric
non_cats = ['ER16515B3', 'ER16515B2', 'ER16515B5', 'ER16515B4',
            'S405', 'S419', 'S420', 'S415', 'S411', 'S413', 'S417', 'S416' ]

non_cats = [c.lower() for c in non_cats]

for c in non_cats:
    try:
        cat.remove(c)
    except KeyError:
        print("Can't remove ", c, " from cat")

    if not c.startswith('S'):
        df[c] = df[c].replace( { 9999999.0: np.nan } )

df['married'] = (df.er33507 != 0)


dfc = cat.to_categorical(df)


In [28]:
# Calculate the sum of all expenditures

# Exclude some to avoid double counts.
excludes = ['er16515a2', 'er16515a3', 'er16515a4', # components of food
            'er16515a6' # mortgage, component of housing

            ]

exp_cols = [c['name'] for c in r.columns() if c['category'] == 'EXPENDITURES' and c['name'] not in excludes ]
dfc['exp'] = dfc[exp_cols].sum(axis=1) # df categorized



In [35]:
col_map = {

    'er30001':'1968_interview_number',
    'er30002':'person_number',
    'er30000':'release_number',

    'er13001': 'release_number',
    'er13009': 'num_in_fu',
    'er13010': 'age_of_head',
    'er13011': 'sex_of_head',
    'er13012': 'age_of_wife',
    'er13013': 'num_children_in_fu',
    'er13041': 'house_value',
    'er15928': 'race_of_head_1',
    'er15929': 'race_of_head_2',
    'er15930': 'race_of_head_3',
    'er15931': 'race_of_head_4',
    'er16462': 'total_family_income',

    's420': 'home_equity',
    's416': 'wealth_wo_equity',
    's417': 'wealth_w_equity',
    'er33501': 'interview_number',
    'er33502': 'sequence_number',
    'er33503': 'relation_to_head',
    'er33507': 'marital_pairs_indicator',
    'er33516': 'years_completed_education',

    'er16519': 'family_weight_cs', # Cross-sectional family weight
    'er16518':'family_weight',

    'er15836':'k34_87_race_of_wife_1',
    'er15837':'k34_87_race_of_wife_2',
    'er15838':'k34_87_race_of_wife_3',
    'er15839':'k34_87_race_of_wife_4',

    'er16515a1': 'food_expenditure',
    'er16515a2': 'food_at_home_expenditure',
    'er16515a3': 'food_away_from_home_expenditure',
    'er16515a4': 'food_delivered_expenditure',
    'er16515a5': 'housing_expenditure',
    'er16515a6': 'mortgage_expenditure',
    'er16515b6': 'transportation_expenditure',
    'er16515b7': 'vehicle_loan_payment_expenditure',
    'er16515b9': 'vehicle_lease_payment_expenditure',
    'er16515c9': 'education_expenditure',
    'er16515d1': 'childcare_expenditure',
    'er16515c2':'additional_vehicle_expenditure',
    'er16515c1':'auto_insurance_expenditure',
    'er16515c6':'bus_train_expenditure',
    'er16515d4':'doctor_expenditure_1998',
    'er16515b3':'electricity_expenditure',
    'er16515b2':'gas_for_home_expenditure',
    'er16515c4':'gasoline_expenditure',
    'er16515d2':'health_care_expenditure',
    'er16515d6':'health_insurance_expenditure',
    'er16515a9':'home_insurance_expenditure',
    'er16515d3':'hospital_nursing_home_expenditure_1998',
    'er16515c8':'other_transportation_expenditure',
    'er16515b5':'other_utility__expenditure',
    'er16515c5':'parking_expenditure',
    'er16515d5':'prescriptions_other_expenditure_1998',
    'er16515a8':'property_tax_expenditure',
    'er16515a7':'rent_expenditure',
    'er16515c7':'taxicab_expenditure',
    'er16515b1':'utility_expenditure',
    'er16515b8':'vehicle_down_payment_expenditure',
    'er16515c3':'vehicle_repair_expenditure',
    'er16515b4':'water_sewer_expenditure'

}

cmap = { c['name']:c for c in r.columns() }

for c in dfc.rename(columns=col_map).columns:
    if  c.startswith('er'):
        print(f"'{c}':'{cmap[c]['label'].replace('/','_').replace(' ','_').replace('_1999','').lower()}',")

dfm = dfc.rename(columns=col_map) # df mapped

In [36]:
cats = dfm.relation_to_head.cat.categories.to_list()
cats[0] = 'na'
cats[1] = 'head'
cats[2] = 'wife'
cats[3] = 'chwife'
cats[4] = 'child_of_head'

dfm['relation_to_head'] = dfm.relation_to_head.cat.rename_categories(cats)

dfm['nonmort_housing_expenditure'] = dfm.housing_expenditure - dfm.mortgage_expenditure



In [37]:
# Remove the records with an inconsistent head of family and fewer than 9 income observations

t = dfm[(dfm.relation_to_head == 'head')].dropna(subset=['total_family_income','race'])
t = t.groupby('pid').total_family_income.count()
valid_pids = t[t==10].index.to_list() # For now, us 10 so we don't have to interpolate

dfh = dfm[(dfm.pid.isin(valid_pids))]

dfh.shape

(31510, 71)

In [38]:
# Add in CPI to adjust for inflation
cpi = pkg.reference('cpi').dataframe()
cpi.columns = ['date','cpi']
cpi['date'] = pd.to_datetime(cpi.date)
cpi.date = pd.to_datetime(cpi.date)
cpi = cpi.groupby(cpi.date.dt.year).mean(numeric_only=True).reset_index().rename(columns={'date':'year'})

cpi = cpi[cpi.year <= dfh.year.max()] # So cpi.max() uses the last year of the dataset

cpi['cpi_current'] = cpi.cpi.max()/cpi.cpi

dfh = dfh.merge(cpi,how='left',on='year')

In [39]:
dfe  = dfh[['pid','year','race','age_of_head', 'num_children_in_fu',
            'total_family_income','exp','home_equity','wealth_w_equity','wealth_wo_equity',
            'housing_expenditure', 'nonmort_housing_expenditure', 'mortgage_expenditure',
            'cpi_current','family_weight'
            ]].copy().sort_values(['pid','year'])

In [40]:
cpi_cols = ['total_family_income','exp','home_equity','wealth_w_equity',  'wealth_wo_equity',
            'nonmort_housing_expenditure', 'mortgage_expenditure']

dfe[cpi_cols] = dfe[cpi_cols].multiply(dfe.cpi_current,axis=0).astype(int)

dfe

,pid,year,race,age_of_head,num_children_in_fu,total_family_income,exp,home_equity,wealth_w_equity,wealth_wo_equity,housing_expenditure,nonmort_housing_expenditure,mortgage_expenditure,cpi_current,family_weight
0,5003,2001,white,38.0,2.0,114366,81581,72200,222379,150178,12416.0,7220,10708,1.444020,22.220
3151,5003,2003,white,41.0,0.0,88227,101838,25009,144498,119489,10806.0,4710,10303,1.389411,23.444
6302,5003,2005,white,42.0,0.0,57606,168324,94265,168892,74626,13746.0,8287,9709,1.309244,23.473
9453,5003,2007,white,45.0,0.0,42661,66551,101104,162753,61649,13366.0,7336,9143,1.232982,25.066
12604,5003,2009,white,46.0,0.0,51234,56514,157276,284766,127489,15974.0,10196,8836,1.191490,25.066
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18905,6872003,2011,black,49.0,2.0,139376,115971,125028,176175,51147,22184.0,11120,14094,1.136619,4.676
22056,6872003,2013,black,51.0,2.0,89527,98234,91087,97453,6365,23740.0,12752,13301,1.097445,3.827
25207,6872003,2015,black,53.0,0.0,58249,65928,97082,97621,539,29184.0,16121,15359,1.078691,2.814
28358,6872003,2017,black,55.0,0.0,82234,55568,140799,141843,1044,21800.0,10429,12306,1.042961,2.814


In [41]:
x = dfe[dfe.pid ==  5003]

def mean_growth(s):
    return ((s-s.shift(1))/s.shift(1)).mean()


(x.total_family_income - x.exp).sum()


-9324

In [45]:
x.home_equity.iloc[-1] - x.home_equity.iloc[0]

127800

In [144]:
# Columns where we will compute a difference between the first and last year
diff_cols = ['home_equity','wealth_w_equity',  'wealth_wo_equity']

# Columns where we will sum the values
sum_cols = ['total_family_income','exp','housing_expenditure', 'nonmort_housing_expenditure', 'mortgage_expenditure']


def compile(x):
    return pd.concat([
        pd.Series({'age':x.age_of_head.mean(), 'race': x.race.mode()[0], 'num_children':x.num_children_in_fu.mean()}),
        x[sum_cols].sum() * 2, # x2 because we only have samples for 1/2 of the years
        x[diff_cols].iloc[-1] - x[diff_cols].iloc[0]
    ])


sumry = dfe[dfe.race.isin(['black','white'])].groupby('pid').apply(compile)


In [145]:
z = sumry.corr(numeric_only=True).stack().reset_index().sort_values(0, ascending=False)
z[z[0]!=1].head(10)

,level_0,level_1,0
89,wealth_w_equity,wealth_wo_equity,0.977504
98,wealth_wo_equity,wealth_w_equity,0.977504
43,housing_expenditure,exp,0.870981
34,exp,housing_expenditure,0.870981
46,housing_expenditure,mortgage_expenditure,0.841173
64,mortgage_expenditure,housing_expenditure,0.841173
45,housing_expenditure,nonmort_housing_expenditure,0.840668
54,nonmort_housing_expenditure,housing_expenditure,0.840668
32,exp,total_family_income,0.776554
23,total_family_income,exp,0.776554


In [146]:
t.corrwith(t.wealth_w_equity, numeric_only=True).sort_values(ascending=False)

wealth_w_equity                1.000000
wealth_wo_equity               0.976539
total_family_income            0.462240
home_equity                    0.356173
exp                            0.321057
housing_expenditure            0.283785
nonmort_housing_expenditure    0.248196
mortgage_expenditure           0.226978
num_children                  -0.005011
age                           -0.037886
dtype: float64

In [147]:
a,b = sumry.total_family_income.quantile([.4,.6])
print(a,b)
sumry_46 = sumry[(sumry.total_family_income >= a) & (sumry.total_family_income <= b)].copy()

1208384.4000000001 1829699.2


In [148]:
sumry_46['wir'] = sumry.wealth_w_equity/sumry.total_family_income
sumry_46['eir'] = sumry.exp/sumry.total_family_income

In [149]:
z = sumry_46.groupby('race').mean()
z['sav'] = z.total_family_income - z.exp
z

,age,num_children,total_family_income,exp,housing_expenditure,nonmort_housing_expenditure,mortgage_expenditure,home_equity,wealth_w_equity,wealth_wo_equity,wir,eir,sav
race,,,,,,,,,,,,,
black,49.401099,0.839560,1.479462e+06,1.411499e+06,325301.449231,263547.021978,115269.879121,30524.412088,9568.379121,-20955.917582,0.004256,0.960822,67962.186813
white,50.564183,0.717308,1.519375e+06,1.412132e+06,311751.596827,247336.504808,115846.394231,43162.987981,82711.550481,39548.853365,0.055607,0.936356,107242.447115


In [ ]:
sumry.describe().T